In [1]:
#create a list of bike station intersections (Van - complete)
#use geocoding to find lat/lng for each bike station (Freeman - complete)
# Convert Zillow API response w/ neighborhood data to json from XML (Arne - complete)

# find a way to identify the neighborhood based on lat/long 

In [2]:
# Dependencies
import json
import requests as req
import pandas as pd
import numpy as np
from xmljson import parker, Parker
from xml.etree.ElementTree import fromstring
import xml.etree.cElementTree as ET
from bs4 import BeautifulSoup
from math import sin, cos, sqrt, atan2, radians

# Save config information
api_key = "X1-ZWz1g5wdt5pyq3_8nu8h"
url = "http://www.zillow.com/webservice/GetRegionChildren.htm?"
city = "Washington"
state = "DC"

# Build query URL
query_url = url + "zws-id=" + api_key + "&city=" + city +"&state=" + state + "&childtype=neighborhood" + "&output=json" 
print(query_url)

# Get Zillow data
zillow_response = req.get(query_url)

with open('zillow_response.xml', 'w') as f:
    f.write(zillow_response.text)
    
def parse_xml(xml):
    soup = BeautifulSoup(xml)
    return soup

if __name__ == '__main__':
    with open('zillow_response.xml', 'r') as zillow_xml:
        xml_contents = zillow_xml.read()
    #soup = parse_xml(xml_contents)
    #print(soup.prettify())
    temp_file = json.dumps(parker.data(fromstring(xml_contents)), indent=4)
    temp2_file = json.loads(temp_file)
    #print(json.dumps(parker.data(fromstring(xml_contents)), indent=4))

temp2_file
#zillow_df = pd.DataFrame(temp2_file)
#zillow_df.head()

http://www.zillow.com/webservice/GetRegionChildren.htm?zws-id=X1-ZWz1g5wdt5pyq3_8nu8h&city=Washington&state=DC&childtype=neighborhood&output=json


{'message': {'code': 0, 'text': 'Request successfully processed'},
 'request': {'childtype': 'neighborhood', 'city': 'Washington', 'state': 'DC'},
 'response': {'list': {'count': 135,
   'region': [{'id': 121697,
     'latitude': 38.928257,
     'longitude': -77.029264,
     'name': 'Columbia Heights',
     'url': 'http://www.zillow.com/local-info/DC-Washington/Columbia-Heights/r_121697/'},
    {'id': 121685,
     'latitude': 38.887156,
     'longitude': -76.998955,
     'name': 'Capitol Hill',
     'url': 'http://www.zillow.com/local-info/DC-Washington/Capitol-Hill/r_121685/'},
    {'id': 121774,
     'latitude': 38.946405,
     'longitude': -77.018236,
     'name': 'Petworth',
     'url': 'http://www.zillow.com/local-info/DC-Washington/Petworth/r_121774/',
     'zindex': 479300},
    {'id': 121668,
     'latitude': 38.92119,
     'longitude': -77.043406,
     'name': 'Adams Morgan',
     'url': 'http://www.zillow.com/local-info/DC-Washington/Adams-Morgan/r_121668/'},
    {'id': 12180

In [3]:
#create a for loop to take key values in the dictionary

zillow_df = pd.DataFrame(temp2_file['response']['list']['region'])
zillow_df["Distance"] = ""
zillow_df.head()

,id,latitude,longitude,name,url,zindex,Distance
0,121697,38.928257,-77.029264,Columbia Heights,http://www.zillow.com/local-info/DC-Washington...,NaN,
1,121685,38.887156,-76.998955,Capitol Hill,http://www.zillow.com/local-info/DC-Washington...,NaN,
2,121774,38.946405,-77.018236,Petworth,http://www.zillow.com/local-info/DC-Washington...,479300.0,
3,121668,38.921190,-77.043406,Adams Morgan,http://www.zillow.com/local-info/DC-Washington...,NaN,
4,121806,38.830750,-76.990587,Washington Highlands,http://www.zillow.com/local-info/DC-Washington...,NaN,


In [4]:
#BRING IN FREEMAN's data to use as to run the loop

#pull in 2010 bike location data to use for mapping
stations = pd.read_csv('https://raw.githubusercontent.com/arnenewman11/project-one/master/CapitalBikeShareDF.csv')
stations = stations[['start_station','start_date']]
stations['Latitude']=''
stations['Longitude']=''
stations['Neighborhood'] = ''
stations['3 months before']=''
stations['3 months after']=''
stations['6 months before']=''
stations['6 months after']=''
stations['start_station'] = stations.apply(lambda x: x.str.replace(' ','+'))
stations.head()


,start_station,start_date,Latitude,Longitude,Neighborhood,3 months before,3 months after,6 months before,6 months after
0,10th+&+e+st+nw,7/12/2012 13:04,,,,,,,
1,10th+&+florida+ave+nw,10/9/2013 8:55,,,,,,,
2,10th+&+g+st+nw,10/1/2016 8:41,,,,,,,
3,10th+&+k+st+nw,7/1/2015 6:32,,,,,,,
4,10th+&+monroe+st+ne,10/16/2010 13:55,,,,,,,


In [5]:
for index, row in stations.iterrows():
    url = 'https://geocoder.cit.api.here.com/6.2/geocode.json'
    app_id = 'YWmtXt51i4qEk4VlEacJ'
    app_code = 'Sna1EDDyBUc7m8EfvgmwMw'
    city = 'Washington'
    street = row['start_station']
    params = {'app_id':app_id,'app_code':app_code,'city':city,'street':street}
    
    response = req.get(url,params).json()
    try:
        lat = response['Response']['View'][0]['Result'][0]['Location']['DisplayPosition']['Latitude']
        lng = response['Response']['View'][0]['Result'][0]['Location']['DisplayPosition']['Longitude']
    
        stations.set_value(index,'Latitude',lat)
        stations.set_value(index,'Longitude',lng)
    except IndexError:
        print('Couldn\'t find coordinates for {0}'.format(street))
    
stations.head()

Couldn't find coordinates for 12th+&+army+navy+dr
Couldn't find coordinates for 15th+&+n+scott+st
Couldn't find coordinates for amherst+ave+&+elkins+st
Couldn't find coordinates for amherst+ave+&+prichard+rd
Couldn't find coordinates for aurora+hills+community+ctr/18th+&+hayes+st
Couldn't find coordinates for ballenger+ave+&+dulaney+st
Couldn't find coordinates for barcroft+community+center
Couldn't find coordinates for battery+ln+&+trolley+trail
Couldn't find coordinates for benning+branch+library
Couldn't find coordinates for bethesda+metro
Couldn't find coordinates for broschart+&+blackwell+rd
Couldn't find coordinates for c+&+o+canal+&+wisconsin+ave+nw
Couldn't find coordinates for congress+heights+metro
Couldn't find coordinates for court+house+metro+/+15th+&+n+uhle+st
Couldn't find coordinates for crabbs+branch+way+&+calhoun+pl
Couldn't find coordinates for crabbs+branch+way+&+redland+rd
Couldn't find coordinates for crystal+city+metro+/+18th+&+bell+st
Couldn't find coordinates f

,start_station,start_date,Latitude,Longitude,Neighborhood,3 months before,3 months after,6 months before,6 months after
0,10th+&+e+st+nw,7/12/2012 13:04,38.6569,-87.1879,,,,,
1,10th+&+florida+ave+nw,10/9/2013 8:55,38.9205,-77.026,,,,,
2,10th+&+g+st+nw,10/1/2016 8:41,38.6569,-87.1879,,,,,
3,10th+&+k+st+nw,7/1/2015 6:32,38.6569,-87.1879,,,,,
4,10th+&+monroe+st+ne,10/16/2010 13:55,38.9324,-76.9959,,,,,


In [6]:
#This is to map the neighborhood to bike shares
#calculate the distance between the neighborhood lat/lon and the bikeshare lat/lon... this needs to be within a loop to calculate
#min distance between each bike share and all neighborhoods

#radius of the earth in km, needed for calculations
R = 6373.0

no_lat_lon = 0

#look through the lat/lon in the full list of unique bike locations: Assume the location 
for index,row in stations.iterrows():

    try:
        lat1 = radians(float(row["Latitude"]))
        lon1 = radians(float(row["Longitude"]))
    except ValueError:
        no_lat_lon += 1
        print("Couldn't convert {}".format(str(row)))
        
        
    #setup a counter
    counter = 0
    
    #setup a list of distances
    distances = []
         
    #loop through all neigborhood locations
    for index,row2 in zillow_df.iterrows():
        
        neighborhood = row2["name"]
              
        #define the lat and lon of the neighborhood location in radians for use in calculations
        lat2 = radians(row2["latitude"])
        lon2 = radians(row2["longitude"])
                
        #calc distance between the bike share lat/lon and neighborhood lat/lon via series of equations found online
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))

        distance = R * c
        print(distance)
        print(neighborhood)
        
        #add distance to the list of distances
        distances.append(distance)
        
        #add the distance to the list of distances
        #row["Distance"] = distance
        
        #change the selected neighborhood if the distance it represents (from the given bike share location)
        # is the shortest distance found yet
        if counter == 0:
            min_distance = distance
            selected_neighborhood = neighborhood
        elif distance < min_distance:
            min_distance = distance
            selected_neighborhood = neighborhood
            
        else:
            continue
    
        #bump up counter
        counter = counter + 1
        
    #add the neighborhood name to the corresponding row in the bike locations database
    
    row["Neighborhood"] = selected_neighborhood

print("We couldn't find lat long for " + str(no_lat_lon) + " stations")

880.7612116554563
Columbia Heights
883.4942725193213
Capitol Hill
881.6740287886397
Petworth
879.5549068793957
Adams Morgan
884.4095654903725
Washington Highlands
880.7239721736023
Logan Circle
879.6490666422955
Dupont Circle
880.5165027352696
Sixteenth Street Heights
877.7868032945729
Georgetown
879.1771783014852
Foggy Bottom
877.7797300660667
Chevy Chase
879.5848412164662
Mount Pleasant
885.7275137122724
Kingman Park
881.0804957188996
Shaw
875.6331954567146
American University Park
880.0253577130123
Brightwood
885.2881299411843
Shipley Terrace
877.8995992906817
Cleveland Park
883.9176058153806
Near Northeast
881.6173503595003
Manor Park
881.8794870647666
Southwest Waterfront
883.1961666272211
Riggs Park
882.94736156574
Catholic University
876.6233909370089
Glover Park
884.6836285223226
Brookland
884.3805960660402
Buena Vista
881.7918316221067
Mount Vernon Square
878.2120931782757
Forest Hills
885.8780476739712
Randle Highlands
888.7590673337385
Benning Ridge
883.5113858834558
Congres

Pleasant Hill
887.6579168473761
Fairfax Village
888.0545819750184
Eastland Gardens
886.5187389281533
Good Hope
890.1943891014615
Burrville
885.8120318437154
Knox Hill
888.4099996381718
Kenilworth
886.7145101405509
National Arboretum
877.7450374884539
Observatory Circle
881.5071986642021
Chinatown
878.4969631110333
Hawthorne
886.4499825828424
Gateway
887.2920858199839
Penn Branch
887.4404492488112
Hillcrest
876.2594774973978
Foxhall
877.9584964517609
Theodore Roosevelt Island
876.5476565433241
Tenleytown
877.2588800092057
Massachusetts Heights
881.6162617111017
Blue Plains Treatment Plant
876.051107852956
Colony Hill
885.6614827941547
Arboretum
880.415064642639
National Mall - West Potomac Park
880.7753864439477
East Potomac Park
878.6644908564996
Lady Bird Johnson Park
2.92633474423972
Columbia Heights
5.036275377004444
Capitol Hill
2.4869104394489248
Petworth
4.29804952762663
Adams Morgan
11.312392659486232
Washington Highlands
3.985439205639714
Logan Circle
4.68663797496906
Dupont Ci

Saint Elizabeths
874.5294067837185
Kent
885.8662499821336
Woodlands
878.2959762965997
Barnaby Woods
877.2781281574838
North Cleveland Park
884.8399579588228
Ivy City
875.4963384429329
Berkley
888.2211725740901
Benning
886.783441917497
Naylor Gardens
882.2849336028355
Anacostia Naval Station - Boiling Air Force Base
889.0932911347844
Benning Heights
882.5304999409796
Sursum Corda Cooperative
888.1811541325621
Mahaning Heights
873.8255717767115
Potomac Heights
890.1911489786524
Capitol View
883.8168172282597
Gallaudet
886.5187806210699
Twining
876.4701310639027
Friendship Heights
876.7693792410289
McLean Gardens
889.7163258249359
Civic Betterment
882.5271994999142
Swampoodle
883.6924102600364
Barry Farm
875.2960427194605
The Palisades
878.4081363758688
Woodland-Normanstone Terrace
881.3294911428676
Penn Quarter
883.6189056685479
Stronghold
883.0280175379352
Pleasant Hill
887.7548739779149
Fairfax Village
888.1514923782108
Eastland Gardens
886.615696853836
Good Hope
890.2913068211832
Burr

883.2930182566213
Riggs Park
883.0442445224747
Catholic University
876.7202897271654
Glover Park
884.7805143615587
Brookland
884.477556318402
Buena Vista
881.8887493855257
Mount Vernon Square
878.3089609281395
Forest Hills
885.9750003916087
Randle Highlands
888.8560060765254
Benning Ridge
883.6083656425919
Congress Heights
884.667846732356
Trinidad
883.2524909935593
Edgewood
882.4395480707102
Bellevue
875.0361102952872
Spring Valley
889.1907425171499
Deanwood
878.4084220591858
Woodley Park
882.3041918001717
Bloomingdale
889.7364304033349
Marshall Heights
888.0231372898115
Fort Dupont
885.5940248320491
Barney Circle
881.5559305764831
Park View
884.2980580005693
Brentwood
884.8871033601772
Anacostia
883.200574739501
Eckington
875.8381480519881
Wesley Heights
885.9222937826037
Woodridge
876.7184845601109
Cathedral Heights
880.6276111902749
Downtown
880.6652992070049
U Street Corridor
883.9994313254277
Michigan Park
883.852353995096
Navy Yard
886.0169135311489
Langston
886.5572323363087
Gr

3.399417837586819
Logan Circle
4.01508969657384
Dupont Circle
3.057958880110079
Sixteenth Street Heights
5.768704661622406
Georgetown
5.313959754886163
Foggy Bottom
6.1074171085297415
Chevy Chase
3.3276693682853846
Mount Pleasant
5.145070574100833
Kingman Park
2.6928538678598843
Shaw
7.424087694147207
American University Park
4.593181944505597
Brightwood
9.915616011815832
Shipley Terrace
5.026245172843226
Cleveland Park
3.3174008455961426
Near Northeast
3.766825238362058
Manor Park
6.741847683978942
Southwest Waterfront
3.4425516344461466
Riggs Park
0.3610176347217438
Catholic University
6.405414887939486
Glover Park
1.7763160339364843
Brookland
7.9218875378399085
Buena Vista
3.385232117498308
Mount Vernon Square
5.051227244093601
Forest Hills
7.563170174417136
Randle Highlands
7.989809266938549
Benning Ridge
10.002465567917298
Congress Heights
3.3258409356917333
Trinidad
1.1162235799075098
Edgewood
12.342393130904584
Bellevue
8.014840034400459
Spring Valley
6.779772641751702
Deanwood


...

Wesley Heights
9.945869615697697
Woodridge
1.2208484624656701
Cathedral Heights
6.239779951128486
Downtown
5.103687148839577
U Street Corridor
8.099070083792908
Michigan Park
10.372005689234689
Navy Yard
10.66856831620947
Langston
12.172510059137903
Greenway
7.241184649244407
Takoma
4.5689595709931465
West End
12.11342385653889
Mayfair
8.581220511077836
North Michigan Park
14.724379968524179
Lincoln Heights
1.974413867351059
Wakefield
13.837548977693677
Garfield Heights
12.27161194866756
River Terrace
10.16956881446727
Carver
13.281862769930044
Douglas
7.201132693798333
Fort Totten
6.785829661714289
Shepherd Park
8.153337142140163
Southwest Federal Center
7.105116374655905
Truxton Circle
5.998007844359922
Brightwood Park
11.50474523154479
Fort Lincoln
15.440113735807351
East Corner
7.820559297916177
NoMa
11.687800652018389
Fairlawn
14.031515230800643
Hillbrook
13.05665276738766
Skyland
9.671012651384153
Langdon
2.590676061780902
Burleith
14.451393148003373
Fort Davis
3.783467172840496


Skyland
663.1532912170577
Langdon
657.1451170135892
Burleith
669.1707721873796
Fort Davis
657.0602673086379
Crestwood
658.6838495579775
Kalorama
660.8259284325067
Ledroit Park
654.2361463073236
Colonial Village
662.4385914014771
Judiciary Square
660.3355632643143
Pleasant Plains
668.3657833890362
Dupont Park
667.1346380746762
Saint Elizabeths
654.525765713233
Kent
668.1287243464872
Woodlands
653.9509999893027
Barnaby Woods
655.4462659247772
North Cleveland Park
663.3886913160212
Ivy City
655.9567525288257
Berkley
667.3387509753509
Benning
668.8255584476481
Naylor Gardens
666.4406070477452
Anacostia Naval Station - Boiling Air Force Base
668.0337821365556
Benning Heights
662.2016165635296
Sursum Corda Cooperative
666.931708825866
Mahaning Heights
653.5077007336747
Potomac Heights
669.1444432053216
Capitol View
662.8941467842246
Gallaudet
667.4287466492635
Twining
653.7054380531948
Friendship Heights
655.6937532410949
McLean Gardens
669.4278036301529
Civic Betterment
662.8605165023881
Sw

379.4088328209932
Forest Hills
370.4780656153521
Randle Highlands
371.98489760044214
Benning Ridge
367.47626751153115
Congress Heights
374.5698224982524
Trinidad
376.37785150009455
Edgewood
365.1538189434499
Bellevue
378.4964458760339
Spring Valley
374.6593686463624
Deanwood
377.12933111513786
Woodley Park
375.7993680914259
Bloomingdale
372.43858611387776
Marshall Heights
372.40953139054113
Fort Dupont
371.98468252200377
Barney Circle
377.53688532560733
Park View
375.8974178522127
Brentwood
369.8926748211346
Anacostia
375.634529477286
Eckington
377.43847477982547
Wesley Heights
377.97964672213243
Woodridge
377.19141945084056
Cathedral Heights
373.9020415257787
Downtown
376.05044377304046
U Street Corridor
379.1400079715834
Michigan Park
371.2499787899605
Navy Yard
374.38778562429366
Langston
371.95939290530936
Greenway
382.6734033141528
Takoma
374.59867545613764
West End
374.3951420611313
Mayfair
379.28719979978945
North Michigan Park
373.43342945367107
Lincoln Heights
379.733406767114

304.61871578590603
Mahaning Heights
293.5674405404921
Potomac Heights
306.6134850980261
Capitol View
301.287846557709
Gallaudet
305.9640985823952
Twining
292.64560856035695
Friendship Heights
295.0802267505395
McLean Gardens
307.176359449047
Civic Betterment
301.8019693358139
Swampoodle
305.8261573232008
Barry Farm
296.217831110929
The Palisades
296.98447937768873
Woodland-Normanstone Terrace
301.19518087323297
Penn Quarter
299.2086727402198
Stronghold
297.6171644734356
Pleasant Hill
307.5568651259369
Fairfax Village
303.59934574292646
Eastland Gardens
307.0007705082576
Good Hope
305.4329862767431
Burrville
307.54038737954295
Knox Hill
303.34188952615943
Kenilworth
302.5631961361819
National Arboretum
296.81066048964965
Observatory Circle
300.9056081067102
Chinatown
291.69269800430436
Hawthorne
301.59068607935757
Gateway
307.161981019117
Penn Branch
307.8654992571201
Hillcrest
297.34625390690644
Foxhall
299.55647989500613
Theodore Roosevelt Island
293.6131972656472
Tenleytown
296.09101

11.164598014325914
Southwest Waterfront
7.286791268063462
Riggs Park
7.431242548320132
Catholic University
4.093169136891947
Glover Park
9.170668700949726
Brookland
13.530696152593528
Buena Vista
8.38258294030437
Mount Vernon Square
2.470202689966607
Forest Hills
13.925068608114255
Randle Highlands
15.246188017283217
Benning Ridge
14.84099399533287
Congress Heights
10.340899906839207
Trinidad
8.207047751433157
Edgewood
16.398913414551497
Bellevue
2.192949987334017
Spring Valley
14.29784451613236
Deanwood
4.0110087133933945
Woodley Park
7.712756134618874
Bloomingdale
15.767906830767615
Marshall Heights
14.317441952257795
Fort Dupont
12.631279393192928
Barney Circle
6.1783399177420515
Park View
9.350176992011601
Brentwood
13.613719528472087
Anacostia
8.540267167203673
Eckington
2.995032659716113
Wesley Heights
10.152464986168104
Woodridge
3.283661797130088
Cathedral Heights
7.8844800142987665
Downtown
6.296104465944127
U Street Corridor
8.049849963385983
Michigan Park
11.923313521595754


In [12]:
stations.to_csv('stations_with_neighborhood.csv', index=False)
stations

,start_station,start_date,Latitude,Longitude,Neighborhood,3 months before,3 months after,6 months before,6 months after
0,10th+&+e+st+nw,7/12/2012 13:04,38.6569,-87.1879,Potomac Heights,,,,
1,10th+&+florida+ave+nw,10/9/2013 8:55,38.9205,-77.026,U Street Corridor,,,,
2,10th+&+g+st+nw,10/1/2016 8:41,38.6569,-87.1879,Potomac Heights,,,,
3,10th+&+k+st+nw,7/1/2015 6:32,38.6569,-87.1879,Potomac Heights,,,,
4,10th+&+monroe+st+ne,10/16/2010 13:55,38.9324,-76.9959,Stronghold,,,,
5,10th+&+u+st+nw,9/20/2010 12:16,38.6569,-87.1879,Potomac Heights,,,,
6,10th+st+&+constitution+ave+nw,10/7/2010 15:42,38.8921,-77.026,Penn Quarter,,,,
7,10th+st+&+l'enfant+plaza+sw,5/19/2016 12:08,38.8836,-77.0249,Southwest Federal Center,,,,
8,11th+&+f+st+nw,2/4/2013 11:24,38.6569,-87.189,Potomac Heights,,,,
9,11th+&+girard+st+nw,6/8/2017 4:32,38.926,-77.0249,Columbia Heights,,,,


In [10]:
zillow_df = pd.read_csv('https://raw.githubusercontent.com/arnenewman11/project-one/master/zillow_neighborhood_ZHVI.csv')
zillow_df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,2001-01,2001-02,2001-03,...,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10
0,121697,Columbia Heights,Washington,DC,Washington,District of Columbia,243,148200.0,149700.0,151200.0,...,593900,600000,603300,597600,589700,585700,585400,583700,582300,582500
1,121685,Capitol Hill,Washington,DC,Washington,District of Columbia,312,210100.0,214300.0,218500.0,...,753500,763700,765300,758800,750500,746100,748000,749300,745500,741400
2,121774,Petworth,Washington,DC,Washington,District of Columbia,490,128200.0,129300.0,130800.0,...,561200,564500,565900,563700,560500,555200,549400,544700,542400,541200
3,121668,Adams Morgan,Washington,DC,Washington,District of Columbia,606,228700.0,231900.0,233800.0,...,562900,568800,570100,568000,566700,566000,565500,564700,562800,561400
4,121806,Washington Highlands,Washington,DC,Washington,District of Columbia,620,94400.0,94200.0,95200.0,...,256500,257500,257600,258800,259200,258800,260500,265400,267600,267600


In [11]:
merge_df = pd.merge(zillow_df, stations, how = 'right', left_on = 'RegionName', right_on = 'Neighborhood')
merge_df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,2001-01,2001-02,2001-03,...,2017-10,start_station,start_date,Latitude,Longitude,Neighborhood,3 months before,3 months after,6 months before,6 months after
0,121697.0,Columbia Heights,Washington,DC,Washington,District of Columbia,243.0,148200.0,149700.0,151200.0,...,582500.0,11th+&+girard+st+nw,6/8/2017 4:32,38.926,-77.0249,Columbia Heights,,,,
1,121697.0,Columbia Heights,Washington,DC,Washington,District of Columbia,243.0,148200.0,149700.0,151200.0,...,582500.0,14th+&+girard+st+nw,5/5/2016 12:30,38.926,-77.0249,Columbia Heights,,,,
2,121697.0,Columbia Heights,Washington,DC,Washington,District of Columbia,243.0,148200.0,149700.0,151200.0,...,582500.0,14th+&+harvard+st+nw,9/20/2010 12:30,38.927,-77.0238,Columbia Heights,,,,
3,121697.0,Columbia Heights,Washington,DC,Washington,District of Columbia,243.0,148200.0,149700.0,151200.0,...,582500.0,14th+&+irving+st+nw,8/2/2016 13:21,38.9286,-77.032,Columbia Heights,,,,
4,121697.0,Columbia Heights,Washington,DC,Washington,District of Columbia,243.0,148200.0,149700.0,151200.0,...,582500.0,14th+st+&+spring+rd+nw,9/20/2010 19:14,38.9371,-77.0327,Columbia Heights,,,,


In [19]:
merge_df = merge_df.dropna(subset=['RegionName'])
merge_df['start_date'] = pd.to_datetime(merge_df.start_date)
merge_df['start_month'] = merge_df['start_date'].dt.strftime('%Y-%m')

merge_df = merge_df.assign(PriceAtInstall = merge_df.lookup(merge_df.index, merge_df.start_month))
merge_df.drop(['RegionID','City','State','Metro','SizeRank','CountyName'],1)
#merge_df.head()

,RegionName,2001-01,2001-02,2001-03,2001-04,2001-05,2001-06,2001-07,2001-08,2001-09,...,start_date,Latitude,Longitude,Neighborhood,3 months before,3 months after,6 months before,6 months after,start_month,PriceAtInstall
0,Columbia Heights,148200.0,149700.0,151200.0,152900.0,154700.0,156700.0,160100.0,164300.0,168300.0,...,2017-06-08 04:32:00,38.926,-77.0249,Columbia Heights,,,,,2017-06,585700.0
1,Capitol Hill,210100.0,214300.0,218500.0,222500.0,229100.0,237800.0,245400.0,251000.0,255800.0,...,2010-10-27 18:30:00,38.8898,-76.9883,Capitol Hill,,,,,2010-10,533500.0
2,Petworth,128200.0,129300.0,130800.0,132600.0,135200.0,138300.0,141300.0,144500.0,147400.0,...,2013-04-12 17:36:00,38.9419,-77.0249,Petworth,,,,,2013-04,365600.0
3,Adams Morgan,228700.0,231900.0,233800.0,236800.0,242000.0,248100.0,253700.0,258700.0,263800.0,...,2013-05-01 07:20:00,38.9187,-77.0417,Adams Morgan,,,,,2013-05,480300.0
4,Logan Circle,186000.0,189100.0,189300.0,190400.0,194100.0,198600.0,203000.0,207300.0,212900.0,...,2010-09-20 17:46:00,38.9088,-77.0319,Logan Circle,,,,,2010-09,439800.0
5,Dupont Circle,152300.0,153400.0,155000.0,157500.0,162300.0,167400.0,171200.0,174800.0,179500.0,...,2010-09-24 10:37:00,38.9119,-77.039,Dupont Circle,,,,,2010-09,353600.0
6,Sixteenth Street Heights,172000.0,172800.0,173800.0,174500.0,177000.0,182900.0,190300.0,196400.0,200600.0,...,2013-04-12 16:37:00,38.9558,-77.0334,Sixteenth Street Heights,,,,,2013-04,504700.0
7,Georgetown,520700.0,523800.0,527400.0,533100.0,543600.0,554000.0,561500.0,568600.0,577200.0,...,2010-10-07 19:02:00,38.9052,-77.0637,Georgetown,,,,,2010-10,878600.0
8,Foggy Bottom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2016-04-19 09:17:00,38.8935,-77.0415,Foggy Bottom,,,,,2016-04,357700.0
9,Mount Pleasant,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2010-10-14 08:15:00,38.9275,-77.0426,Mount Pleasant,,,,,2010-10,608100.0
